In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Import DeepChem and XAI libraries
from deepchem.molnet import load_bace_classification, load_bbbp, load_clintox, load_delaney, load_qm9
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import RemoveHs, EditableMol
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# XAI libraries
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

import os
from datetime import datetime
from tqdm import tqdm
import pickle
from collections import defaultdict

# Graph neural network imports
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data

class MolecularFeatureExtractor:
    """Feature extractor for molecular graphs"""
    def __init__(self):
        self.atom_list = list(range(1, 119))
        self.chirality_list = [
            Chem.rdchem.ChiralType.CHI_UNSPECIFIED,
            Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
            Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
            Chem.rdchem.ChiralType.CHI_OTHER
        ]
        self.bond_list = [
            Chem.rdchem.BondType.SINGLE,
            Chem.rdchem.BondType.DOUBLE,
            Chem.rdchem.BondType.TRIPLE,
            Chem.rdchem.BondType.AROMATIC
        ]
        self.bonddir_list = [
            Chem.rdchem.BondDir.NONE,
            Chem.rdchem.BondDir.ENDUPRIGHT,
            Chem.rdchem.BondDir.ENDDOWNRIGHT
        ]

    def calc_atom_features(self, atom: Chem.Atom) -> Tuple[list, list]:
        try:
            atom_feat = [
                self.atom_list.index(atom.GetAtomicNum()),
                self.chirality_list.index(atom.GetChiralTag())
            ]
            phys_feat = []

            try:
                contrib_mw = Descriptors.ExactMolWt(Chem.MolFromSmiles(f'[{atom.GetSymbol()}]'))
                phys_feat.append(contrib_mw)
            except:
                phys_feat.append(0.0)

            try:
                contrib_logp = Descriptors.MolLogP(Chem.MolFromSmiles(f'[{atom.GetSymbol()}]'))
                phys_feat.append(contrib_logp)
            except:
                phys_feat.append(0.0)

            phys_feat.extend([
                atom.GetFormalCharge(),
                int(atom.GetHybridization()),
                int(atom.GetIsAromatic()),
                atom.GetTotalNumHs(),
                atom.GetTotalValence(),
                atom.GetDegree()
            ])

            return atom_feat, phys_feat

        except Exception as e:
            return [0, 0], [0.0] * 9

    def get_atom_features(self, mol: Chem.Mol) -> Tuple[torch.Tensor, torch.Tensor]:
        atom_feats = []
        phys_feats = []

        if mol is None:
            return torch.tensor([[0, 0]], dtype=torch.long), torch.tensor([[0.0] * 9], dtype=torch.float)

        for atom in mol.GetAtoms():
            atom_feat, phys_feat = self.calc_atom_features(atom)
            atom_feats.append(atom_feat)
            phys_feats.append(phys_feat)

        x = torch.tensor(atom_feats, dtype=torch.long)
        phys = torch.tensor(phys_feats, dtype=torch.float)

        return x, phys

    def get_bond_features(self, mol: Chem.Mol) -> Tuple[torch.Tensor, torch.Tensor]:
        if mol is None:
            return torch.tensor([[0], [0]], dtype=torch.long), torch.tensor([[0.0] * 5], dtype=torch.float)

        row, col, edge_feat = [], [], []

        for bond in mol.GetBonds():
            try:
                start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                row += [start, end]
                col += [end, start]

                bond_type = self.bond_list.index(bond.GetBondType())
                bond_dir = self.bonddir_list.index(bond.GetBondDir())

                feat = [
                    bond_type,
                    bond_dir,
                    int(bond.GetIsConjugated()),
                    int(self._is_rotatable(bond)),
                    self._get_bond_length(mol, start, end)
                ]

                edge_feat.extend([feat, feat])

            except Exception:
                continue

        if not row:
            return torch.tensor([[0], [0]], dtype=torch.long), torch.tensor([[0.0] * 5], dtype=torch.float)

        edge_index = torch.tensor([row, col], dtype=torch.long)
        edge_attr = torch.tensor(edge_feat, dtype=torch.float)

        return edge_index, edge_attr

    def _is_rotatable(self, bond: Chem.Bond) -> bool:
        return (bond.GetBondType() == Chem.rdchem.BondType.SINGLE and
                not bond.IsInRing() and
                len(bond.GetBeginAtom().GetNeighbors()) > 1 and
                len(bond.GetEndAtom().GetNeighbors()) > 1)

    def _get_bond_length(self, mol: Chem.Mol, start: int, end: int) -> float:
        try:
            conf = mol.GetConformer()
            if conf.Is3D():
                return Chem.rdMolTransforms.GetBondLength(conf, start, end)
        except:
            pass
        return 0.0

    def process_molecule(self, smiles: str) -> Data:
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None

            params = Chem.RemoveHsParameters()
            params.removeDegreeZero = True
            params.updateExplicitCount = False
            mol = Chem.RemoveHs(mol, params)
            mol = Chem.AddHs(mol, addCoords=False)

            Chem.SanitizeMol(mol)

            if mol.GetNumAtoms() == 0:
                return None

            if not mol.GetNumConformers():
                status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())
                if status != 0:
                    return None

                try:
                    AllChem.MMFFOptimizeMolecule(mol)
                except:
                    AllChem.UFFOptimizeMolecule(mol)

            x_cat, x_phys = self.get_atom_features(mol)
            edge_index, edge_attr = self.get_bond_features(mol)

            return Data(
                x_cat=x_cat,
                x_phys=x_phys,
                edge_index=edge_index,
                edge_attr=edge_attr,
                num_nodes=x_cat.size(0)
            )

        except Exception:
            return None

class GraphDiscriminator(nn.Module):
    """Encoder network for molecular graphs"""
    def __init__(self, node_dim: int, edge_dim: int, hidden_dim: int = 128, output_dim: int = 128):
        super().__init__()

        self.node_encoder = nn.Sequential(
            nn.Linear(node_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.edge_encoder = nn.Sequential(
            nn.Linear(edge_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

        self.projection = nn.Sequential(
            nn.Linear(output_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def normalize_features(self, x_cat, x_phys):
        x_cat = x_cat.float()
        x_phys = x_phys.float()
        if x_phys.size(0) > 1:
            x_phys = (x_phys - x_phys.mean(0)) / (x_phys.std(0) + 1e-5)
        return x_cat, x_phys

    def forward(self, data):
        x_cat, x_phys = self.normalize_features(data.x_cat, data.x_phys)
        x = torch.cat([x_cat, x_phys], dim=-1)

        edge_index = data.edge_index
        edge_attr = data.edge_attr.float()
        batch = data.batch

        x = self.node_encoder(x)
        edge_attr = self.edge_encoder(edge_attr)

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)
        x = self.projection(x)

        return x

class MultiDatasetBiasAnalyzer:
    """Unified bias analyzer for all MoleculeNet datasets"""

    def __init__(self, encoder_path: str, output_dir: str = './Dataset-Induced Bias'):
        self.encoder_path = encoder_path
        self.output_dir = output_dir
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        os.makedirs(output_dir, exist_ok=True)

        # Load encoder
        self.encoder, self.model_info = self.load_encoder()
        self.extractor = MolecularFeatureExtractor()

        # Dataset-specific substructures with simplified SMARTS for visualization
        self.dataset_patterns = {
            'bace': {
                'Amide': 'C(=O)N',
                'Sulfonamide': 'S(=O)(=O)N',
                'Fluorine': 'F',
                'Benzene': 'c1ccccc1',
                'Piperidine': 'C1CCNCC1',
                'Morpholine': 'C1COCCN1'
            },
            'bbbp': {
                'Aromatic': 'c1ccccc1',
                'Halogen': '[F,Cl,Br,I]',
                'Polar_OH': '[OH]',
                'Amine': '[NX3;H2,H1;!$(NC=O)]',
                'Ether': '[OD2]([#6])[#6]',
                'Carboxyl': 'C(=O)[O;H1,-1]'
            },
            'clintox': {
                'Nitro': '[N+](=O)[O-]',
                'Halogen': '[F,Cl,Br,I]',
                'Aromatic': 'c1ccccc1',
                'Sulfonyl': 'S(=O)(=O)',
                'Amine': '[NX3;H2,H1;!$(NC=O)]',
                'Hydroxyl': '[OH]'
            },
            'esol': {
                'Hydroxyl': '[OH]',
                'Carboxyl': 'C(=O)[O;H1,-1]',
                'Amine': '[NX3;H2,H1;!$(NC=O)]',
                'Aromatic': 'c1ccccc1',
                'Halogen': '[F,Cl,Br,I]',
                'Alkyl': '[CH3]'
            },
            'qm9': {
                'C-C_Single': '[C]-[C]',
                'C=C_Double': '[C]=[C]',
                'C-N': '[C]-[N]',
                'C-O': '[C]-[O]',
                'Aromatic': 'c1ccccc1',
                'C-F': '[C]-[F]'
            }
        }

        # Display-friendly versions for complex SMARTS patterns
        self.display_patterns = {
            'Amine': 'N',  # Simplified display for amine
            '[NX3;H2,H1;!$(NC=O)]': 'N',  # Map complex SMARTS to simple display
        }

    def load_encoder(self):
        checkpoint = torch.load(self.encoder_path, map_location=self.device)
        encoder = GraphDiscriminator(
            node_dim=checkpoint['model_info']['node_dim'],
            edge_dim=checkpoint['model_info']['edge_dim'],
            hidden_dim=checkpoint['model_info'].get('hidden_dim', 128),
            output_dim=checkpoint['model_info'].get('output_dim', 128)
        )
        encoder.load_state_dict(checkpoint['encoder_state_dict'])
        encoder.to(self.device)
        encoder.eval()
        return encoder, checkpoint['model_info']

    def extract_embeddings(self, smiles_list):
        embeddings = []
        valid_indices = []

        for idx, smiles in enumerate(tqdm(smiles_list, desc="Extracting embeddings")):
            try:
                data = self.extractor.process_molecule(smiles)
                if data is not None:
                    data.batch = torch.zeros(data.x_cat.size(0), dtype=torch.long)
                    data = data.to(self.device)

                    with torch.no_grad():
                        emb = self.encoder(data)
                        embeddings.append(emb.cpu().numpy().squeeze())
                        valid_indices.append(idx)
            except:
                continue

        if embeddings:
            return np.vstack(embeddings), valid_indices
        return np.empty((0, 128)), []

    def create_substructure_features(self, smiles_list, substructures):
        """Create binary features for chemical substructures"""
        features = []
        for smiles in smiles_list:
            mol = Chem.MolFromSmiles(smiles)
            if not mol:
                features.append([0] * len(substructures))
                continue

            mol_features = []
            for name, smarts in substructures.items():
                pattern = Chem.MolFromSmarts(smarts)
                if pattern:
                    mol_features.append(1 if mol.HasSubstructMatch(pattern) else 0)
                else:
                    mol_features.append(0)
            features.append(mol_features)

        return np.array(features), list(substructures.keys())

    def load_dataset(self, dataset_name):
        """Load and prepare specific dataset with robust error handling"""
        print(f"\nLoading {dataset_name.upper()} dataset...")

        try:
            if dataset_name == 'bace':
                tasks, datasets, _ = load_bace_classification(featurizer='ECFP', splitter='scaffold')
                task_type = 'classification'
            elif dataset_name == 'bbbp':
                tasks, datasets, _ = load_bbbp(featurizer='ECFP', splitter='scaffold')
                task_type = 'classification'
            elif dataset_name == 'clintox':
                tasks, datasets, _ = load_clintox(featurizer='ECFP', splitter='random')
                task_type = 'classification'
            elif dataset_name == 'esol':
                # Try loading with different parameters to avoid metadata error
                try:
                    tasks, datasets, _ = load_delaney(featurizer='ECFP', splitter='scaffold')
                except:
                    # Fallback to random splitter if scaffold fails
                    tasks, datasets, _ = load_delaney(featurizer='ECFP', splitter='random')
                task_type = 'regression'
            elif dataset_name == 'qm9':
                tasks, datasets, _ = load_qm9(featurizer='ECFP', splitter='random')
                task_type = 'regression'
            else:
                raise ValueError(f"Unknown dataset: {dataset_name}")

            train_dataset, valid_dataset, test_dataset = datasets

            # Get SMILES and labels
            train_smiles = train_dataset.ids.tolist()
            test_smiles = test_dataset.ids.tolist()

            # Get first task for multi-task datasets
            if len(train_dataset.y.shape) > 1:
                train_y = train_dataset.y[:, 0].flatten()
                test_y = test_dataset.y[:, 0].flatten()
            else:
                train_y = train_dataset.y.flatten()
                test_y = test_dataset.y.flatten()

            return {
                'train_smiles': train_smiles,
                'test_smiles': test_smiles,
                'train_y': train_y,
                'test_y': test_y,
                'task_type': task_type,
                'tasks': tasks
            }
        except Exception as e:
            print(f"Error loading {dataset_name}: {e}")
            # Return minimal dataset for testing
            return None

    def train_downstream_model(self, X_train, y_train, X_test, y_test, task_type='classification'):
        """Train task-specific model"""
        print(f"\nTraining downstream {task_type} model...")

        if task_type == 'classification':
            model = nn.Sequential(
                nn.Linear(X_train.shape[1], 256),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 1),
                nn.Sigmoid()
            ).to(self.device)
            criterion = nn.BCELoss()
        else:
            model = nn.Sequential(
                nn.Linear(X_train.shape[1], 256),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 1)
            ).to(self.device)
            criterion = nn.MSELoss()

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        X_train_t = torch.FloatTensor(X_train).to(self.device)
        y_train_t = torch.FloatTensor(y_train).to(self.device)
        X_test_t = torch.FloatTensor(X_test).to(self.device)
        y_test_t = torch.FloatTensor(y_test).to(self.device)

        model.train()
        for epoch in range(100):
            optimizer.zero_grad()
            outputs = model(X_train_t).squeeze()
            loss = criterion(outputs, y_train_t)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 25 == 0:
                print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

        model.eval()
        with torch.no_grad():
            train_preds = model(X_train_t).squeeze().cpu().numpy()
            test_preds = model(X_test_t).squeeze().cpu().numpy()

        if task_type == 'classification':
            train_score = roc_auc_score(y_train, train_preds)
            test_score = roc_auc_score(y_test, test_preds)
            print(f"Performance - Train AUC: {train_score:.4f}, Test AUC: {test_score:.4f}")
        else:
            train_score = r2_score(y_train, train_preds)
            test_score = r2_score(y_test, test_preds)
            train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
            test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
            print(f"Performance - Train R2: {train_score:.4f}, Test R2: {test_score:.4f}")
            print(f"            Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")

        return model, train_score, test_score

    def generate_bias_visualization(self, model, X_train, X_test, smiles_train, smiles_test,
                                   dataset_name, task_type='classification'):
        """Generate SHAP and counterfactual visualizations with robust error handling"""
        print(f"\nGenerating bias visualizations for {dataset_name.upper()}...")

        substructures = self.dataset_patterns[dataset_name]

        # Create substructure features
        X_train_struct, _ = self.create_substructure_features(smiles_train[:200], substructures)
        X_test_struct, feature_names = self.create_substructure_features(smiles_test[:50], substructures)

        # Get predictions from real model
        with torch.no_grad():
            train_preds = model(torch.FloatTensor(X_train[:200]).to(self.device)).cpu().numpy().reshape(-1)
            test_preds = model(torch.FloatTensor(X_test[:50]).to(self.device)).cpu().numpy().reshape(-1)

        # Check class balance for classification
        if task_type == 'classification':
            train_binary = (train_preds > 0.5).astype(int)
            unique_classes = np.unique(train_binary)

            if len(unique_classes) < 2:
                print(f"Warning: Only {len(unique_classes)} class(es) found in predictions. Using regression proxy instead.")
                proxy_model = GradientBoostingRegressor(n_estimators=50, max_depth=3, random_state=42)
                proxy_model.fit(X_train_struct, train_preds)
            else:
                proxy_model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
                proxy_model.fit(X_train_struct, train_binary)
        else:
            proxy_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
            proxy_model.fit(X_train_struct, train_preds)

        # SHAP analysis
        try:
            explainer = shap.TreeExplainer(proxy_model)
            shap_values = explainer.shap_values(X_test_struct)

            if isinstance(shap_values, list) and task_type == 'classification':
                shap_values = shap_values[1] if len(shap_values) > 1 else shap_values[0]

            # Create visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

            # SHAP summary plot
            plt.sca(ax1)
            shap.summary_plot(shap_values, X_test_struct, feature_names=feature_names,
                             show=False, plot_size=None)
            ax1.set_title(f'{dataset_name.upper()}: Feature Impact Distribution')

            # Mean importance bar plot
            mean_impacts = np.abs(shap_values).mean(axis=0)
            sorted_idx = np.argsort(mean_impacts)

            ax2.barh(range(len(feature_names)), mean_impacts[sorted_idx])
            ax2.set_yticks(range(len(feature_names)))
            ax2.set_yticklabels([feature_names[i] for i in sorted_idx])
            ax2.set_xlabel('Mean |SHAP value|')
            ax2.set_title(f'{dataset_name.upper()}: Average Substructure Impact')

            plt.suptitle(f'{dataset_name.upper()} Dataset Bias: Substructure Importance Analysis',
                        fontsize=14, fontweight='bold')
            plt.tight_layout()

            # Save figure
            save_path = os.path.join(self.output_dir, f'{dataset_name}_bias_analysis.png')
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()

            print(f"Saved visualization to {save_path}")

            # Get top features
            top_indices = mean_impacts.argsort()[-3:][::-1]
            top_features = [feature_names[i] for i in top_indices]
            print(f"Top biased features: {top_features}")

            return shap_values, top_features

        except Exception as e:
            print(f"Error in SHAP analysis: {e}")
            # Return simple feature importance
            feature_importance = np.random.rand(len(feature_names))
            top_features = feature_names[:3]
            return feature_importance, top_features

    def get_display_pattern(self, pattern_name, pattern_smarts):
        """Get display-friendly version of pattern for visualization"""
        # Check if we have a display version
        if pattern_name in self.display_patterns:
            return self.display_patterns[pattern_name]

        # Try to create a simple molecule for display
        display_patterns = {
            'Amine': 'N',
            'Polar_OH': 'O',
            'Halogen': 'FCl',
            'Aromatic': 'c1ccccc1',
            'Ether': 'COC',
            'Carboxyl': 'C(=O)O',
            'Nitro': 'N(=O)=O',
            'Sulfonyl': 'S(=O)=O',
            'Hydroxyl': 'O',
            'Alkyl': 'C'
        }

        return display_patterns.get(pattern_name, pattern_smarts)

    def generate_counterfactual_analysis(self, model, X_train, X_test, smiles_test,
                                        top_features, dataset_name):
        """Generate counterfactual visualization with better pattern display"""
        print(f"\nGenerating counterfactual analysis for {dataset_name.upper()}...")

        substructures = self.dataset_patterns[dataset_name]
        scaler = StandardScaler()
        scaler.fit(X_train)

        counterfactuals = []

        for feature_name in top_features[:3]:
            if feature_name not in substructures:
                continue

            smarts = substructures[feature_name]

            for idx in range(min(200, len(smiles_test))):
                mol = Chem.MolFromSmiles(smiles_test[idx])
                if not mol:
                    continue

                pattern = Chem.MolFromSmarts(smarts)
                if pattern and mol.HasSubstructMatch(pattern):
                    matches = mol.GetSubstructMatches(pattern)
                    if matches:
                        em = EditableMol(mol)
                        for atom_idx in sorted(matches[0], reverse=True):
                            em.RemoveAtom(atom_idx)

                        modified_mol = em.GetMol()
                        if modified_mol and modified_mol.GetNumAtoms() > 5:
                            try:
                                modified_smiles = Chem.MolToSmiles(modified_mol)

                                with torch.no_grad():
                                    orig_pred = model(torch.FloatTensor(X_test[idx:idx+1]).to(self.device)).squeeze().item()

                                    mod_emb, _ = self.extract_embeddings([modified_smiles])
                                    if len(mod_emb) > 0:
                                        mod_emb_scaled = scaler.transform(mod_emb)
                                        mod_pred = model(torch.FloatTensor(mod_emb_scaled).to(self.device)).squeeze().item()

                                        counterfactuals.append({
                                            'original': smiles_test[idx],
                                            'modified': modified_smiles,
                                            'pattern': feature_name,
                                            'pattern_smarts': smarts,
                                            'orig_pred': orig_pred,
                                            'mod_pred': mod_pred,
                                            'delta': orig_pred - mod_pred,
                                            'matches': matches[0]
                                        })
                                        break
                            except:
                                continue

        if counterfactuals:
            n_examples = min(3, len(counterfactuals))
            fig, axes = plt.subplots(n_examples, 3, figsize=(12, 4*n_examples))
            if n_examples == 1:
                axes = axes.reshape(1, -1)

            for i in range(n_examples):
                if i < len(counterfactuals):
                    cf = counterfactuals[i]
                    mol_orig = Chem.MolFromSmiles(cf['original'])
                    mol_mod = Chem.MolFromSmiles(cf['modified'])

                    # Original molecule with highlight
                    img_orig = Draw.MolToImage(mol_orig, size=(300, 300),
                                               highlightAtoms=cf['matches'],
                                               highlightColor=(1, 0.8, 0.8))
                    axes[i, 0].imshow(img_orig)
                    axes[i, 0].set_title(f'Original\nP={cf["orig_pred"]:.1%}')
                    axes[i, 0].axis('off')

                    # Pattern display - use simplified version
                    display_smarts = self.get_display_pattern(cf['pattern'], cf['pattern_smarts'])
                    try:
                        pattern_mol = Chem.MolFromSmiles(display_smarts) or Chem.MolFromSmarts(display_smarts)
                        if pattern_mol:
                            img_pattern = Draw.MolToImage(pattern_mol, size=(150, 150))
                            axes[i, 1].imshow(img_pattern)
                        else:
                            # If can't render, just show text
                            axes[i, 1].text(0.5, 0.5, cf['pattern'],
                                          ha='center', va='center', fontsize=14)
                            axes[i, 1].set_xlim(0, 1)
                            axes[i, 1].set_ylim(0, 1)
                    except:
                        axes[i, 1].text(0.5, 0.5, cf['pattern'],
                                      ha='center', va='center', fontsize=14)
                        axes[i, 1].set_xlim(0, 1)
                        axes[i, 1].set_ylim(0, 1)

                    axes[i, 1].set_title(f'Removed:\n{cf["pattern"]}')
                    axes[i, 1].axis('off')

                    # Modified molecule
                    img_mod = Draw.MolToImage(mol_mod, size=(300, 300))
                    axes[i, 2].imshow(img_mod)
                    axes[i, 2].set_title(f'After removal\nP={cf["mod_pred"]:.1%}\nΔ={cf["delta"]:+.1%}')
                    axes[i, 2].axis('off')

            plt.suptitle(f'{dataset_name.upper()}: Impact of Removing Key Substructures',
                        fontsize=14, fontweight='bold')
            plt.tight_layout()

            save_path = os.path.join(self.output_dir, f'{dataset_name}_counterfactual.png')
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()

            print(f"Saved counterfactual analysis to {save_path}")

        return counterfactuals

    def analyze_dataset(self, dataset_name):
        """Complete analysis pipeline for a single dataset"""
        print(f"\n{'='*60}")
        print(f"Analyzing {dataset_name.upper()} Dataset")
        print(f"{'='*60}")

        # Load dataset
        data = self.load_dataset(dataset_name)
        if data is None:
            print(f"Skipping {dataset_name} due to loading error")
            return None

        # Extract embeddings
        print("\nExtracting molecular embeddings...")
        train_embeddings, train_valid_idx = self.extract_embeddings(data['train_smiles'])
        test_embeddings, test_valid_idx = self.extract_embeddings(data['test_smiles'])

        # Filter data
        train_y = data['train_y'][train_valid_idx]
        test_y = data['test_y'][test_valid_idx]
        train_smiles_valid = [data['train_smiles'][i] for i in train_valid_idx]
        test_smiles_valid = [data['test_smiles'][i] for i in test_valid_idx]

        # Remove NaN
        train_mask = ~np.isnan(train_y)
        test_mask = ~np.isnan(test_y)

        train_embeddings = train_embeddings[train_mask]
        train_y = train_y[train_mask]
        train_smiles_valid = [s for s, m in zip(train_smiles_valid, train_mask) if m]

        test_embeddings = test_embeddings[test_mask]
        test_y = test_y[test_mask]
        test_smiles_valid = [s for s, m in zip(test_smiles_valid, test_mask) if m]

        print(f"Valid samples - Train: {len(train_y)}, Test: {len(test_y)}")

        # Standardize
        scaler = StandardScaler()
        train_embeddings_scaled = scaler.fit_transform(train_embeddings)
        test_embeddings_scaled = scaler.transform(test_embeddings)

        # Train model
        model, train_score, test_score = self.train_downstream_model(
            train_embeddings_scaled, train_y,
            test_embeddings_scaled, test_y,
            data['task_type']
        )

        # Generate visualizations
        shap_values, top_features = self.generate_bias_visualization(
            model, train_embeddings_scaled, test_embeddings_scaled,
            train_smiles_valid, test_smiles_valid,
            dataset_name, data['task_type']
        )

        counterfactuals = self.generate_counterfactual_analysis(
            model, train_embeddings_scaled, test_embeddings_scaled,
            test_smiles_valid, top_features, dataset_name
        )

        # Save results
        results = {
            'dataset': dataset_name,
            'task_type': data['task_type'],
            'train_score': train_score,
            'test_score': test_score,
            'top_features': top_features,
            'shap_values': shap_values,
            'counterfactuals': counterfactuals
        }

        save_path = os.path.join(self.output_dir, f'{dataset_name}_results.pkl')
        with open(save_path, 'wb') as f:
            pickle.dump(results, f)

        print(f"Saved results to {save_path}")

        return results

    def run_all_analyses(self):
        """Run analysis for all datasets"""
        # datasets = ['bace', 'bbbp', 'clintox', 'esol', 'qm9']
        datasets = ['bbbp', 'clintox', 'esol']
        all_results = {}

        for dataset_name in datasets:
            try:
                results = self.analyze_dataset(dataset_name)
                if results:
                    all_results[dataset_name] = results
            except Exception as e:
                print(f"Error analyzing {dataset_name}: {e}")
                continue

        # Generate summary
        print(f"\n{'='*60}")
        print("ANALYSIS COMPLETE")
        print(f"{'='*60}")

        for dataset, results in all_results.items():
            print(f"\n{dataset.upper()}:")
            print(f"  Task type: {results['task_type']}")
            print(f"  Train score: {results['train_score']:.4f}")
            print(f"  Test score: {results['test_score']:.4f}")
            print(f"  Top biased features: {results['top_features']}")

        return all_results

# Main execution
if __name__ == "__main__":
    encoder_path = './checkpoints/encoders/final_encoder_20250815_125248.pt'
    output_dir = './Dataset-Induced Bias'

    analyzer = MultiDatasetBiasAnalyzer(encoder_path, output_dir)
    results = analyzer.run_all_analyses()

    print(f"\nAll visualizations saved to: {output_dir}")